In [33]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains 

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from PIL import Image
from io import BytesIO
import time
import csv
import os
import base64


In [4]:
import selenium
print("versión selenium:", selenium.__version__)

versión selenium: 4.19.0


In [26]:
# Nombre del archivo CSV
denomination_file = "-actas-vigentes.csv"
denomination_images = "-images"
# Ruta completa al archivo CSV en el directorio "data" dentro del directorio actual
initial_data = ["NRO ACTA", 
                "TITULAR", 
                "FECHA INGRESO",
                "CLASE",
                "DENOMINACION",
                "TIPO DE MARCA",
                "NRO RESOLUCION",
                "ESTADO",
                "VENCIMIENTO",
                "PROTECCION",
                "LIMITACION",
                "NOMBRE IMAGEN"]

def routeFile(number):
    name = "class" + str(number) + denomination_file
    return os.path.join(os.getcwd(), "data", name)

def routeFolderImages(number):
    name = "class" + str(number) + denomination_images
    return os.path.join(os.getcwd(), "data", name)

def createFile(number):
    with open(routeFile(number), mode='w', newline='') as archivo_csv:
        escritor_csv = csv.writer(archivo_csv)
        escritor_csv.writerow(initial_data)

def appendLines(number, data):
    with open(routeFile(number), mode='a', newline='') as archivo_csv:
        escritor_csv = csv.writer(archivo_csv)
                
        for line in data:
            escritor_csv.writerow(line)

def storeImage(number, url, act_number):
    route = routeFolderImages(number)
    base64_image = url.split(",")[1]
    image_bytes = base64.b64decode(base64_image)
    image = Image.open(BytesIO(image_bytes))
    if not os.path.exists(route):
        os.makedirs(route)
    image.save(os.path.join(route, act_number + ".jpg"))




createFile(2)



In [40]:
def botClass(number):
    # Funciones
    def makeCell(line):
        aLine = []
        for cell in line:
            aLine.append(cell.text)
        aLine.pop()
        return aLine

    def wait_for_element_to_hide(driver, locator, timeout=10):
        WebDriverWait(driver, timeout).until(
            EC.invisibility_of_element_located((locator))
        )
    
    # Opciones de navegacion
    print("Open Search for class ", number)
    options = Options()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-extensions')

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get('https://portaltramites.inpi.gob.ar/marcasconsultas/busqueda/?Cod_Funcion=NQA0ADEA')

    time.sleep(1)

    collapsables = driver.find_elements(By.CLASS_NAME, "accordion-toggle");
    desiredText = "BUSCADOR"
    for collapsable in collapsables:
        if desiredText in collapsable.text:
            collapsable.click()
           
    time.sleep(1)

    class_dropdown = Select(driver.find_element(By.ID, "clase"));
    class_dropdown.select_by_value(str(number))

    checkbox = driver.find_element(By.CLASS_NAME, "glyphicon-ok")
    checkbox.click()
    createFile(number)
    print("Create folder for class", number)
    
    search_button = driver.find_element(By.ID, "BtnBuscarAvanzada")
    search_button.click()
    print("Changing page in class", number)
    wait_for_element_to_hide(driver, (By.CSS_SELECTOR, ".fixed-table-loading"), timeout=60)
    time.sleep(1)


    print("loading first entry for class", number)
    tab_original = driver.current_window_handle
    last_page_number = driver.find_element(By.CLASS_NAME, "page-last").text

    for i in range(1, int(last_page_number)):
        for n in range(0,10):
            table = driver.find_element(By.ID, "tblGrillaMarcas")
            tbody = table.find_element(By.TAG_NAME, 'tbody')
            tableElements = tbody.find_elements(By.TAG_NAME, "tr")
            detailPage = tableElements[n].find_element(By.CLASS_NAME, "fa-plus-square")

            line = makeCell(tableElements[n].find_elements(By.TAG_NAME, "td"))
            hover = ActionChains(driver).move_to_element(detailPage)
            hover.perform()

            detailPage.click() 

            p = driver.current_window_handle
            chwd = driver.window_handles
            for w in chwd:
                if(w!=p):
                    driver.switch_to.window(w)

            time.sleep(4)
            data = driver.find_element(By.ID, "collapse-two")
            labels = data.find_elements(By.TAG_NAME, "label")
            for label in labels:
                if "PROTECCION" in label.text:
                    try:
                        line.append(label.find_element(By.TAG_NAME, "span").text)
                    except:
                        line.append("")
                if "LIMITACION" in label.text:
                    try:
                        line.append(label.text.split(": ")[1])
                    except:
                        line.append("")

            try:
                imageId = driver.find_element(By.ID, "logo")  
                image = imageId.find_element(By.TAG_NAME, "img")
                image_url = image.get_attribute("src")
                storeImage(number=number, url=image_url, act_number=line[0])
                line.append(line[0] + ".jpg")
            except:
                line.append("")

            
            driver.close()
            appendLines(number=number, data=[line])
            time.sleep(1)
            driver.switch_to.window(tab_original)
        print("I finished i page for class", number)
        next = driver.find_element(By.CLASS_NAME, "page-next")
        clickeablePage = next.find_element(By.TAG_NAME, "a")
        clickeablePage.click()
        wait_for_element_to_hide(driver, (By.CSS_SELECTOR, ".fixed-table-loading"), timeout=60)
        time.sleep(1)
        driver.switch_to.window(tab_original)

In [42]:
import concurrent.futures

def classSearcher(number):
    print(f"Ejecutando classSearcher({number})")
    botClass(str(number))
    print(f"Finalizando classSearcher({number})")

veces_totales = 45
veces_en_paralelo = 10

with concurrent.futures.ThreadPoolExecutor(max_workers=veces_en_paralelo) as executor:
    for i in range(1, veces_totales + 1, veces_en_paralelo):
        numeros = range(i, min(i + veces_en_paralelo, veces_totales + 1))
        futures = [executor.submit(classSearcher, num) for num in numeros]
        concurrent.futures.wait(futures)





Ejecutando classSearcher(1)
Open Search for class  1
Ejecutando classSearcher(2)
Open Search for class  2
Ejecutando classSearcher(3)
Open Search for class  3
Ejecutando classSearcher(4)
Open Search for class  4
Ejecutando classSearcher(5)
Open Search for class  5
Ejecutando classSearcher(6)
Open Search for class  6
Ejecutando classSearcher(7)
Open Search for class  7
Ejecutando classSearcher(8)
Open Search for class  8
Ejecutando classSearcher(9)
Open Search for class  9
Ejecutando classSearcher(10)
Open Search for class  10
Create folder for class 1
Changing page in class 1
Create folder for class 3
Changing page in class 3
Create folder for class 6
Changing page in class 6
Create folder for class 5
loading first entry for class 1
Create folder for class 2
Create folder for class 9
Create folder for class 4
Create folder for class 8
Create folder for class 10
Create folder for class 7
Changing page in class 4
Changing page in class 2
Changing page in class 10
Changing page in class 9

KeyboardInterrupt: 

In [ ]:
    #time.sleep(1)
    #date_before = form.find_element(By.ID, "IngresoDesde");
    #date_after = form.find_element(By.ID, "IngresoHasta");
    #date_before.send_keys("01/01/2020")
    #date_after.send_keys("01/01/2113")

    #pageSize = driver.find_element(By.CLASS_NAME, "page-size")
    #pageSize.click()
    #time.sleep(1)

    #pageList = driver.find_element(By.CLASS_NAME, "page-list")
    #btnGroup = pageList.find_element(By.CLASS_NAME, "btn-group")
    #DropdownMenu = btnGroup.find_elements(By.CLASS_NAME, "dropdown-menu")

    #for optionMenu in DropdownMenu:
    #    if "200" in str(optionMenu.text):
    #        optionMenu.click()
    #time.sleep(11)